# Accenture Job Simulation: Data Analytics and Visualization

## Project Overview
In this project, I will analyze SocialBuzz's content categories to identify the top 5 categories with the highest popularity.

## Import required libraries and datasets

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
content = pd.read_csv(r'D:\REGINA\DA DS\accenture_data_analysis\Content.csv', index_col=0)
reactions = pd.read_csv(r'D:\REGINA\DA DS\accenture_data_analysis\Reactions.csv', index_col=0)
reactiontypes = pd.read_csv(r'D:\REGINA\DA DS\accenture_data_analysis\ReactionTypes.csv', index_col=0)

In [3]:
content.head(5)

,Content ID,User ID,Type,Category,URL
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,photo,Studying,https://socialbuzz.cdn.com/content/storage/975...
1,9f737e0a-3cdd-4d29-9d24-753f4e3be810,beb1f34e-7870-46d6-9fc7-2e12eb83ce43,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/9f7...
2,230c4e4d-70c3-461d-b42c-ec09396efb3f,a5c65404-5894-4b87-82f2-d787cbee86b4,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/230...
3,356fff80-da4d-4785-9f43-bc1261031dc6,9fb4ce88-fac1-406c-8544-1a899cee7aaf,photo,technology,https://socialbuzz.cdn.com/content/storage/356...
4,01ab84dd-6364-4236-abbb-3f237db77180,e206e31b-5f85-4964-b6ea-d7ee5324def1,video,food,https://socialbuzz.cdn.com/content/storage/01a...


In [4]:
reactions.head(5)

,Content ID,User ID,Type,Datetime
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,NaN,NaN,2021-04-22 15:17:15
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,5d454588-283d-459d-915d-c48a2cb4c27f,disgust,2020-11-07 09:43:50
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,92b87fa5-f271-43e0-af66-84fac21052e6,dislike,2021-06-17 12:22:51
3,97522e57-d9ab-4bd6-97bf-c24d952602d2,163daa38-8b77-48c9-9af6-37a6c1447ac2,scared,2021-04-18 05:13:58
4,97522e57-d9ab-4bd6-97bf-c24d952602d2,34e8add9-0206-47fd-a501-037b994650a2,disgust,2021-01-06 19:13:01


In [5]:
reactiontypes.head(5)

,Type,Sentiment,Score
0,heart,positive,60
1,want,positive,70
2,disgust,negative,0
3,hate,negative,5
4,interested,positive,30


## Data Cleaning

### Removing irrelevant columns

In [6]:
content = content.drop(columns=['User ID', 'URL'])
reactions = reactions.drop(columns=['User ID'])

### Replacing column names

In [7]:
content = content.rename(columns={'Type':'Content Type'})
reactions = reactions.rename(columns={'Type':'Reaction Type'})
reactiontypes = reactiontypes.rename(columns={'Type':'Reaction Type'})

### Removing rows with missing values

In [8]:
content.isna().sum()

Content ID      0
Content Type    0
Category        0
dtype: int64

In [9]:
reactiontypes.isna().sum()

Reaction Type    0
Sentiment        0
Score            0
dtype: int64

In [10]:
reactions.isna().sum()

Content ID         0
Reaction Type    980
Datetime           0
dtype: int64

In [11]:
reactions = reactions.dropna().reset_index()

### Changing the data type of specific values in a column

In [12]:
content['Category'].unique()

array(['Studying', 'healthy eating', 'technology', 'food', 'cooking',
       'dogs', 'soccer', 'public speaking', 'science', 'tennis', 'travel',
       'fitness', 'education', 'studying', 'veganism', 'Animals',
       'animals', 'culture', '"culture"', 'Fitness', '"studying"',
       'Veganism', '"animals"', 'Travel', '"soccer"', 'Education',
       '"dogs"', 'Technology', 'Soccer', '"tennis"', 'Culture', '"food"',
       'Food', '"technology"', 'Healthy Eating', '"cooking"', 'Science',
       '"public speaking"', '"veganism"', 'Public Speaking', '"science"'],
      dtype=object)

There is some inconsistent data in the column. Some values have quotation marks and start with a capital letter.

In [13]:
content['Category'] = content['Category'].str.replace('"','').str.lower()

## Data Modelling

### Merging the three tables together

In [15]:
data = reactions.merge(content, on='Content ID', how='inner')
data = data.merge(reactiontypes, on='Reaction Type', how='inner')
data.head()

,index,Content ID,Reaction Type,Datetime,Content Type,Category,Sentiment,Score
0,1,97522e57-d9ab-4bd6-97bf-c24d952602d2,disgust,2020-11-07 09:43:50,photo,studying,negative,0
1,2,97522e57-d9ab-4bd6-97bf-c24d952602d2,dislike,2021-06-17 12:22:51,photo,studying,negative,10
2,3,97522e57-d9ab-4bd6-97bf-c24d952602d2,scared,2021-04-18 05:13:58,photo,studying,negative,15
3,4,97522e57-d9ab-4bd6-97bf-c24d952602d2,disgust,2021-01-06 19:13:01,photo,studying,negative,0
4,5,97522e57-d9ab-4bd6-97bf-c24d952602d2,interested,2020-08-23 12:25:58,photo,studying,positive,30


### Find the Top 5 performing categories

In [20]:
top5 = data.groupby('Category')['Score'].sum().reset_index().sort_values('Score', ascending=False).head(5)
top5

,Category,Score
0,animals,74965
9,science,71168
7,healthy eating,69339
12,technology,68738
6,food,66676


## Export tables into Excel

In [25]:
file_path = r'D:\REGINA\DA DS\accenture_data_analysis\cleandata.xlsx'
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    content.to_excel(writer, sheet_name='content', index=False)
    reactions.to_excel(writer, sheet_name='reactions', index=False)
    reactiontypes.to_excel(writer, sheet_name='reactiontypes', index=False)
    data.to_excel(writer, sheet_name='finaldata', index=False)
    top5.to_excel(writer, sheet_name='top5', index=False)